In [1]:
import os
import sys

from RFEM.enums import LineSupportType, SurfaceGeometry, SurfaceLoadDirection, SurfaceLoadDistribution, ActionCategoryType
from RFEM.initModel import Calculate_all, Model
from RFEM.BasicObjects.node import Node
from RFEM.BasicObjects.line import Line
from RFEM.BasicObjects.material import Material
from RFEM.BasicObjects.thickness import Thickness
from RFEM.BasicObjects.surface import Surface
from RFEM.TypesForLines.lineSupport import LineSupport
from RFEM.TypesForNodes.nodalSupport import NodalSupport
from RFEM.Results.resultTables import GetMaxValue, ResultTables
from RFEM.Loads.surfaceLoad import SurfaceLoad
from RFEM.LoadCasesAndCombinations.loadCase import LoadCase
from RFEM.ImportExport.exports import ExportTo

Model(True, 'ChatGPT_Tank.rf6')
Model.clientModel.service.begin_modification()

Connecting to server...


# GENERAL PROMPT

#### I want to create a cylindrical steel pressure vessel with a diameter of 2m, heigt 5 m and an upper lid under snow load on the top. The support is at the bottom circular line. the thickness of the lid is 10 mm and for the cylinder shell 4 mm. the magnitude of the snow load is 0.25 kN/m².

# MATERIAL

In [2]:
Material(1, "S235")

# GEOMETRY

In [3]:
# Define nodes for the bottom and top circles
Node(1, 0, 0, 0)  # Center of the bottom circle
Node(2, 1, 0, 0)  # Point on the perimeter of the bottom circle
Node(3, 0, 0, -5) # Center of the top circle
Node(4, 1, 0, -5) # Point on the perimeter of the top circle

# Define the bottom circle
Line.Circle(1, [0, 0, 0], 1, [0, 0, 1])

# Define the top circle
Line.Circle(2, [0, 0, -5], 1, [0, 0, 1])

# Define the vertical lines connecting the top and bottom circles    
Line(3, '2 4')  # Line from bottom perimeter to top perimeter        

# Define the support at the bottom circle
LineSupport(1, '1', LineSupportType.HINGED)

# CROSSSECTION

In [4]:
# Define the thickness for the lid and the tank wall
Thickness(1, '5 mm Lid', 1, 0.005)  # 5 mm thickness for the lid     
Thickness(2, '3 mm Wall', 1, 0.003)  # 3 mm thickness for the tank wall

# Define the surfaces for the lid and the tank wall
Surface.Standard(1, SurfaceGeometry.GEOMETRY_PLANE, None, '2', 1)  # Top lid surface using the top circle
Surface.Standard(2, SurfaceGeometry.GEOMETRY_ROTATED, [360, [0, 0, 0], [0, 0, 1], 3], '3', 2)  # Cylindrical wall surface using the vertical line

# LOADS

In [5]:
# Define the load case without self-weight
LoadCase(1, 'Snow Load', self_weight=[False])

# Define the snow load on the top lid surface
SurfaceLoad.Force(
    no=1,
    load_case_no=1,
    surface_no='1',  # Top lid surface
    load_direction=SurfaceLoadDirection.LOAD_DIRECTION_GLOBAL_Z_OR_USER_DEFINED_W_TRUE,
    load_distribution=SurfaceLoadDistribution.LOAD_DISTRIBUTION_UNIFORM,
    load_parameter=[0.25 * 1000]  # Convert kN/m² to N/m²
)

# APPLY CALL

In [6]:
Model.clientModel.service.finish_modification()

# Calculate Call

In [7]:
Calculate_all()

# GET RESULTS

In [8]:
global maxStress
maxStressinit = ResultTables.SurfacesEquivalentStressesMises()
maxStress = GetMaxValue(maxStressinit, 'equivalent_stresses_sigma_eqv_max') / 1000000

maxStress

16.647792